In [1]:
%matplotlib inline
from nuscenes.nuscenes import NuScenes
import nuscenes.lidarseg
from nuscenes.utils.data_classes import LidarPointCloud, RadarPointCloud, Box
import utils_data_viewer
from utils_data_viewer import view_scene_map, custom_draw_geometry, cam_vis, top_down, interactive_vis, visualize_lidar_in_img
from utils_data_viewer import project_points_in_img, colorise_pcd, visualize_lidar_in_img_sweeps, visualize_radar_in_img_sweeps
#Radar
from utils_data_viewer import top_down_radar, interactive_vis_radar, radar_from_file, fuse_radars_in_ego, visualize_radar_in_img
import utils_obj_det
from utils_obj_det import get_radar_in_img, vis_obj_det_speed

import numpy as np
import os
import time
from scipy.spatial.transform import Rotation
from pyquaternion import Quaternion
import struct
import matplotlib as plt

import cv2
import open3d as o3d


nusc = NuScenes(version='v1.0-mini', dataroot='/data/sets/nuscenes', verbose=True)
utils_data_viewer.nusc = nusc
utils_obj_det.nusc = nusc

#nusc.list_scenes()
sc_id = 4

scene = nusc.scene[sc_id]
log = nusc.get('log', scene['log_token'])
map = nusc.get('map', log['map_token'])
map_file = os.path.join(nusc.dataroot, map['filename'])

first_sample = nusc.get('sample', scene['first_sample_token'])
last_sample = nusc.get('sample', scene['last_sample_token'])
sample = first_sample

Jupyter environment detected. Enabling Open3D WebVisualizer.
[Open3D INFO] WebRTC GUI backend enabled.
[Open3D INFO] WebRTCWindowSystem: HTTP handshake server disabled.
Loading NuScenes tables for version v1.0-mini...
23 category,
8 attribute,
4 visibility,
911 instance,
12 sensor,
120 calibrated_sensor,
31206 ego_pose,
8 log,
10 scene,
404 sample,
31206 sample_data,
18538 sample_annotation,
4 map,
Done loading in 0.467 seconds.
Reverse indexing ...
Done reverse indexing in 0.1 seconds.


In [2]:
first_sample = nusc.get('sample', scene['first_sample_token'])
last_sample = nusc.get('sample', scene['last_sample_token'])
sample = first_sample
cam = nusc.get('sample_data', sample['data']['CAM_FRONT'])
cam_sensor = nusc.get('calibrated_sensor', cam['calibrated_sensor_token'])
lidar = nusc.get('sample_data', first_sample['data']['LIDAR_TOP'])

In [ ]:
#cam_vis_sweep(scene, 'CAM_FRONT')
#visualize_lidar_in_img_sweeps(scene, cam_channel='CAM_BACK')
visualize_radar_in_img_sweeps(scene, cam_channel='CAM_FRONT')

### Record

In [ ]:
# sc_ids = [0, 2, 3, 4, 6, 7, 8]
# channels = ['CAM_FRONT', 'CAM_FRONT_LEFT', 'CAM_FRONT_RIGHT', 'CAM_BACK', 'CAM_BACK_LEFT', 'CAM_BACK_RIGHT']
# for sc_i in sc_ids:
#     scene = nusc.scene[sc_i]
#     for c in channels:
    #     cam_vis(scene, c, record=True)
    #     visualize_lidar_in_img(scene, c, record=True)
    #     visualize_radar_in_img(scene, c, record=True)
    #     vis_obj_det_speed(scene, cam_channel=c)
    # top_down(scene, record=True)
    #top_down_radar(scene, record=True)
    # interactive_vis(scene, record=True)
    # colorise_pcd(scene, record=True)
    # interactive_vis_radar(scene, record=True)
    

# Visualize

In [ ]:
#nusc.render_pointcloud_in_image(sample['token'], pointsensor_channel='LIDAR_TOP')

In [ ]:
#Lidar
rec = False
#view_scene_map(scene)
#visualize_lidar_in_img(scene, cam_channel='CAM_FRONT')
#colorise_pcd(scene, record=rec)
#cam_vis(scene, "CAM_BACK")
#interactive_vis(scene, record=rec)
#top_down(scene, record=True)

#Radar
#interactive_vis_radar(scene, record=True)
#interactive_vis_radar_sweeps(scene)
#top_down_radar(scene, record=True)
#visualize_radar_in_img(scene, cam_channel='CAM_BACK', record=True)

### BACKUP

In [ ]:
radar_channel = 'RADAR_FRONT'
radar = nusc.get('sample_data', sample['data'][radar_channel])
radar_file = os.path.join(nusc.dataroot, radar['filename'])
pts, fields = radar_from_file(radar_file)
radar_sensor = nusc.get('calibrated_sensor', radar['calibrated_sensor_token'])
velocities = pts[8:9, :]  # Compensated velocity

s, f = fuse_radars_in_ego(sample)
s.shape

velocities = np.vstack((velocities, np.zeros(pts.shape[1])))
velocities.shape

In [ ]:
np.expand_dims(pts, 1)

# End RADAR

In [ ]:
#visualize_lidar_in_img(scene, cam_channel='CAM_FRONT')
#colorise_pcd(scene)

In [ ]:
#Show the map of the scene
#view_scene_map(scene)
#visualize_lidar_in_img(scene, cam_channel='CAM_FRONT')
#colorise_pcd(scene)
#cam_vis(scene, "CAM_BACK")
interactive_vis(scene, record=True)
#top_down(scene, record=True)

In [ ]:
#Show front cam view through scene
#cam_vis(scene, "CAM_BACK")

### Show Pointcloud LIDAR

In [ ]:
#Visualize with interactive view control
#interactive_vis(scene, record=True)

In [ ]:
#Visualize top down view only
#top_down(scene, record=True)

### Fuse colored pointcloud

In [ ]:
first_sample = nusc.get('sample', scene['first_sample_token'])
last_sample = nusc.get('sample', scene['last_sample_token'])
sample = first_sample

In [ ]:
first_sample = nusc.get('sample', scene['first_sample_token'])
last_sample = nusc.get('sample', scene['last_sample_token'])
sample = first_sample
channels = ['CAM_FRONT', 'CAM_FRONT_LEFT', 'CAM_FRONT_RIGHT', 'CAM_BACK', 'CAM_BACK_LEFT', 'CAM_BACK_RIGHT']

pcd = o3d.geometry.PointCloud()
#Visualize a video sequence of LIDAR data  

#Render callback for non blocking visualization which allows smoother control of view
#Define the callback here to simplify the transfer of variables samples, and pcd
def render_callback(vis):
    global sample, pcd, channels
    cam_channel = 'CAM_FRONT_LEFT'
    if sample['token'] == last_sample['token']:
        return False

    #Load next scan
    #cam = nusc.get('sample_data', sample['data'][cam_channel])
    #Load lidar scan
    lidar = nusc.get('sample_data', sample['data']['LIDAR_TOP'])

    #Merged points
    pcd_points =np.empty((0, 3))
    pcd_colors =np.empty((0, 3))
    for channel in channels:
        #Load next scan
        cam = nusc.get('sample_data', sample['data'][channel])
        points, color = project_points_in_img(cam, lidar)
        pcd_points = np.append(pcd_points, points, axis=0)
        pcd_colors = np.append(pcd_colors, color, axis=0)

    #pcd_points, pcd_color = project_points_in_img(cam, lidar)
    #####
    print("FINAL POINTS: ", pcd_points.shape)
    lidar_file = nusc.get_sample_data_path(lidar['token'])
    scan = np.fromfile(lidar_file, dtype=np.float32).reshape(-1, 5) #x, y, z, intensity, Ring Index
    points = scan[:, :3]
    #print("ORIG Points: ", points.shape)
    #print("Percentage: ", pcd_points.shape[0] / points.shape[0])
    #####
    #Update Geometry
    if sample['token'] == first_sample['token']:
        pcd.points = o3d.utility.Vector3dVector(pcd_points)
        pcd.colors = o3d.utility.Vector3dVector(pcd_colors)
        vis.add_geometry(pcd)
    pcd.points = o3d.utility.Vector3dVector(pcd_points) #Only need XYZ
    pcd.colors = o3d.utility.Vector3dVector(pcd_colors)
    #vis.clear_geometries()
    #vis.add_geometry(pcd)

    # Update the visualization window and process events to handle keyboard inputs.
    vis.update_geometry(pcd)
    vis.poll_events()
    vis.update_renderer()

    #Next Sample
    sample = nusc.get('sample', sample['next'])

    #Slow down animation
    time.sleep(0.25)

    return True


# Create an Open3D visualization window.
vis = o3d.visualization.VisualizerWithKeyCallback()

# Set the custom update function.
vis.register_animation_callback(render_callback)


# Start animation.
vis.create_window()

opt = vis.get_render_option()
opt.background_color = np.asarray([0, 0, 0])
opt.point_size = 2.5 #Original 5

vis.run()
vis.destroy_window()

In [ ]:
pcd_points =np.empty((3, 0))

In [ ]:
np.append(pcd_points, np.array([[1, 1], [2, 2], [3, 3] ]), axis=1)

### LIDAR 2 IMG Projection

In [ ]:
#Load lidar and img data from scan
sample = first_sample
cam_channel = 'CAM_FRONT_RIGHT'
#Load camera image
cam = nusc.get('sample_data', sample['data'][cam_channel])
cam_file = os.path.join(nusc.dataroot, cam['filename'])
cam_img = cv2.imread(cam_file)

#Load lidar scan
lidar = nusc.get('sample_data', sample['data']['LIDAR_TOP'])
lidar_file = nusc.get_sample_data_path(lidar['token'])
scan = np.fromfile(lidar_file, dtype=np.float32).reshape(-1, 5) #x, y, z, intensity, Ring Index
points = scan[:, :3] #Only x, y, z

#Get Lidar Pose
lidar_sensor = nusc.get('calibrated_sensor', lidar['calibrated_sensor_token'])
lidar_t = np.asarray(lidar_sensor['translation'])
lidar_r_quat = Quaternion(lidar_sensor['rotation'])

#Get Camera Pose and intrinsic parameters
cam_sensor = nusc.get('calibrated_sensor', cam['calibrated_sensor_token'])
cam_t = np.asarray(cam_sensor['translation'])
cam_r_quat = Quaternion(cam_sensor['rotation'])
cam_k = np.asarray(cam_sensor['camera_intrinsic'])

#Transform points to image frame - Using Open3D transformations
pcd = o3d.geometry.PointCloud()
pcd.points = o3d.utility.Vector3dVector(points)

#First transformation
pcd.rotate(lidar_r_quat.rotation_matrix, np.array([0, 0, 0]))
pcd.translate(lidar_t)

#Second transformation
pcd.translate(-cam_t)
pcd.rotate(cam_r_quat.rotation_matrix.T, np.array([0, 0, 0]))

#Project points into image plane
view = np.copy(cam_k)
points = np.asarray(pcd.points).T[:3, :]

#Set color to depth value
depth = points[2, :]

#Prepare intrinsics matrix for homogenous transformation
viewpad = np.eye(4)
viewpad[:view.shape[0], :view.shape[1]] = view

#Prepare points for homogenous transformation
points = np.concatenate((points, np.ones((1, points.shape[1])))) #4 X N matrix - Homogenous

#Project Points into image
points = np.dot(viewpad, points)

#Remove w
points = points[:3, :]

#Normalize along Z axis - Divide by depth
points = points / points[2, :]

# Remove points that are either outside or behind the camera. Leave a margin of 1 pixel for aesthetic reasons.
# Also make sure points are at least 1m in front of the camera to avoid seeing the lidar points on the camera
# casing for non-keyframes which are slightly out of sync.
min_dist = 1.0
mask = np.ones(points.shape[1], dtype=bool)
mask = np.logical_and(mask, depth > min_dist)
mask = np.logical_and(mask, points[0, :] > 1)
mask = np.logical_and(mask, points[0, :] < cam_img.shape[1] - 1)#cv image has x axis on dimension 1
mask = np.logical_and(mask, points[1, :] > 1)
mask = np.logical_and(mask, points[1, :] < cam_img.shape[0] - 1)
points_img = np.uint8(np.rint(points))[:2, mask] #The x, y coordinates of the points that lie in the image
points = points[:, mask]

#Extract color values from image
pts_color = np.zeros((points.shape[1], 3))
for i in range(points.shape[1]):
    pts_color[i] = cam_img[int(np.rint(points[1, i])), int(np.rint(points[0, i]))]
#pts_color = (pts_color - pts_color.min()) / (pts_color.max() - pts_color.min()) #Normalize to range 0 to 1

#BGR to RGB
pts_color = pts_color[:, ::-1]
pts_color = pts_color.astype(np.float32) / 255.0

pcd_points = scan[mask, :3]
pcd = o3d.geometry.PointCloud()
pcd.points = o3d.utility.Vector3dVector(pcd_points) #Nx3 #pcd_points
pcd.colors = o3d.utility.Vector3dVector(pts_color) #Nx3

#o3d.visualization.draw_geometries([pcd])
custom_draw_geometry(pcd)

In [ ]:
#Load lidar scan
lidar = nusc.get('sample_data', sample['data']['LIDAR_TOP'])
lidar_file = nusc.get_sample_data_path(lidar['token'])
scan = np.fromfile(lidar_file, dtype=np.float32).reshape(-1, 5) #x, y, z, intensity, Ring Index
pcd_points = scan[:, :3] #Only x, y, z

#pcd_points = scan[mask, :3]
pcd = o3d.geometry.PointCloud()
pcd.points = o3d.utility.Vector3dVector(pcd_points) #Nx3 #pcd_points
#pcd.colors = o3d.utility.Vector3dVector(pts_color) #Nx3

o3d.visualization.draw_geometries([pcd])
#custom_draw_geometry(pcd)

In [ ]:

winname = cam['channel']
window_width, window_height = 800, 600
cv2.namedWindow(winname, cv2.WINDOW_NORMAL)
cv2.resizeWindow(winname, window_width, window_height)

cv2.imshow(winname, cam_img)
cv2.waitKey(0)
cv2.destroyAllWindows()

In [ ]:
cv2.imwrite('output_image.jpg', cam_img)


In [ ]:
cam_img.shape